In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1a2.dev20
cdv, version 1.1.1
Python 3.10.6


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000

In [3]:
# from setup-sim notebook
# alice 4070016934
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])

In [4]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch194n24esv92hzz574frun8wsegt4vyv3ncsp5hyleav42c6dm0kmse727kn'

# Create Alice's Waiting Room Coin

In [5]:
%%bash
chia wallet send -f 4070016934 -t txch194n24esv92hzz574frun8wsegt4vyv3ncsp5hyleav42c6dm0kmse727kn -a 1.000000000001

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 4070016934 -tx 0x9a21ed76562ac49439ab14475b4165ea43999d97e7f97b127067405f5318cfa9' to get status


In [6]:
%%bash
cdv decode txch194n24esv92hzz574frun8wsegt4vyv3ncsp5hyleav42c6dm0kmse727kn

2d66aae60c2aae2153d548f933ba1942eac23233c4034b93f9eb2aac69bb7db7


In [7]:
%%bash
cdv rpc coinrecords --by puzzlehash "2d66aae60c2aae2153d548f933ba1942eac23233c4034b93f9eb2aac69bb7db7"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x2d66aae60c2aae2153d548f933ba1942eac23233c4034b93f9eb2aac69bb7db7"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1664270246
    }
]


## Clawback

In [9]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 10

Farmed 10 Transaction blocks
Block Height is now: 16


## Clawback is not available yet

In [10]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_puzzle_coin = coin_records[0].coin
alice_waiting_room_puzzle_coin

Coin { parent_coin_info: df2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b, puzzle_hash: 2d66aae60c2aae2153d548f933ba1942eac23233c4034b93f9eb2aac69bb7db7, amount: 1000000000001 }